In [13]:
'''
***************
  INSTRUCTION
***************

This file is used to deal with 10k cat dataset coded with python3
The whole dataset is divided into two dataset folders.
The directory structure is shown below:
----CAT_DATASET_01
        ---CAT_00
        ---CAT_01
        ---CAT_02
----CAT_DATASET_02
        ---CAT_03
        ---CAT_04
        ---CAT_05
        ---CAT_06

If you want to run this file, you shoul make sure two things:
1. the file should be at the same directory as CAT_DATASET_01 and CAT_DATASET_01 folders
2. You should install package PIL first

There are some other details you need to deal with when you use this code to deal with dataset_01 and dataset_02
The instruction will be written with the execution part of the code
'''
from PIL import Image
import os

In [9]:
'''
*********************
 FUNCTION DEFINITION
*********************
This part of code contains all the functions will be used.
'''
def getFileNames(batch,DataSetNum):
    # os.getcwd(): get current directory
    # get all the file name ends with "jpg", store in Cat_jpg
    # get all the file name ends with "cat", store in Cat_jpg
    Cat_jpg = [f for f in os.listdir(os.getcwd()+'/CAT_DATASET_0'+ str(DataSetNum) +'/CAT_0'+str(batch)) if f.endswith('.jpg')]
    Cat_Coor = [f for f in os.listdir(os.getcwd()+'/CAT_DATASET_0'+ str(DataSetNum) +'/CAT_0' + str(batch)) if f.endswith('.cat')]
    return [Cat_jpg, Cat_Coor]

def readCoorStrTxt(filename,batchNum,DataSetNum):
    # The corrdinates of each part of cats are store in .cat file
    # This function read the coordinates into a str s and return it
    f = open(os.getcwd()+'/CAT_DATASET_0'+ str(DataSetNum) +'/CAT_0'+str(batchNum)+'/' + filename)
    for line in f:
        s = line
    return s

def ParseCoorsFromStr(coorStr):
    # parse the coordinates from str to int and return
    return [int(x) for x in coorStr.split(" ")[:-1]]


def getSlope(Coor):
    # calculate the slope of coordinate of cat's two eyes.
    # If the eyes are at the same column, return -1
    x = abs(Coor[3] - Coor[1])
    y = abs(Coor[4] - Coor[2])
    if x == 0:
        return -1
    return y / x

def getCropPoints(Coor):
    # get the arguments for PIL's crop function
    return [Coor[9],min(Coor[10],Coor[16]),Coor[15],int(Coor[6] + abs(Coor[6] - Coor[16])*(1/3))]
def getImage(filename, batch,DataSetNum):
    # Load image with the batch number and filename
    return Image.open(os.getcwd()+'/CAT_DATASET_0'+ str(DataSetNum) +'/CAT_0' + str(batch) +'/'+filename)
    # Save image, create folder if not exist
def saveImg(img,filename, batch,DataSetNum):
    dir =os.path.dirname(os.getcwd()+'/CAT_DATASET_0'+ str(DataSetNum) +'/OUTPUT_'+str(batch)+'/')
    if not os.path.exists(dir):
        os.makedirs(dir)
    img.save(os.getcwd()+'/CAT_DATASET_0'+ str(DataSetNum) +'/OUTPUT_'+str(batch)+'/'+filename)

In [11]:
'''
****************
 EXECUTION PART
****************
There is only one variable need to be assigned manually, which is DataSetNum
As told by instruction, the whole dataset are divided into two parts,
If you want to get the cat face of the first part of data set, DataSetNum1.
Otherwise, DataSetNum = 2
'''
DataSetNum = 1
upper = 1.4
lower = 0.7
lrange = 0
hrange = 3
if DataSetNum == 2:
    lrange = 3
    hrange = 3
for batch in range(lrange,hrange):
    Filenames = getFileNames(batch,DataSetNum)
    # get the file name list end with '.jpg' and '.cat' separately.
    cat_jpg = Filenames[0]
    cat_coor = Filenames[1]
    # Use two pointer method to iterate all files
    jpg_count = 0
    cat_count = 0
    # 
    while jpg_count < len(cat_jpg) and cat_count <len(cat_coor):
        # Compare the file name of the file name (ignore '.jpg' for jpg file and '.jpg.cat' for coordinate file)
        # if the filename of two files are the same, these two file match
        # if not, select to move one pointer by comparing their filename(smaller one move pointer)
        if cat_jpg[jpg_count][:-4] < cat_coor[cat_count][:-8]:
            jpg_count += 1
        elif cat_jpg[jpg_count][:-4] > cat_coor[cat_count][:-8]:
            cat_count += 1
        elif cat_jpg[jpg_count][:-4] == cat_coor[cat_count][:-8]:
            jpg_count += 1
            cat_count += 1
            coorStr = readCoorStrTxt(cat_coor[cat_count - 1],batch,DataSetNum)
            coorList = ParseCoorsFromStr(coorStr)
            slope = getSlope(coorList)
            # if the slope is too big, ignore this image
            if slope <= 0.2 and slope != -1:
                img_origin = getImage(cat_jpg[jpg_count - 1],batch,DataSetNum)
                CropPoints = getCropPoints(coorList)
                # if the propotion of two images is out of a range, ignore it.
                # Because a normal cat face should be approximately a square
                if not CropPoints[2] <= CropPoints[0] and not CropPoints[3] <= CropPoints[1]:
                    temp = abs(CropPoints[2] - CropPoints[0])/abs(CropPoints[3] - CropPoints[1])
                    if lower < temp and upper > temp:
                        box = (CropPoints[0],CropPoints[1],CropPoints[2],CropPoints[3])
                        output_img = img_origin.crop(box)
                        saveImg(output_img,cat_jpg[jpg_count - 1],batch,DataSetNum)
                